# Global Registration using Fast Point Feature Histograms (FPFH)

In [37]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import copy

In [39]:
# source = o3d.io.read_point_cloud("./Data/output/bottle/pcd/bottle_left.ply")
# target = o3d.io.read_point_cloud("./Data/output/bottle/pcd/bottle_right.ply")
source = o3d.io.read_point_cloud("./Data/output/house/pcd/IMG_2630.ply")
target = o3d.io.read_point_cloud("./Data/output/house/pcd/IMG_2631.ply")

Function to display results

In [40]:
def draw_registration_result(source, target, transformation, diff_color=True):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if diff_color:
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

To reduce the computational impact we downsample the point clouds and use a voxel grid to reduce the number of points.

In [41]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal) 
    #Noramls are used to calculate the FPFH features
    pcd_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [42]:
voxel_size=0.00001
# random transformation matrix
trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                            [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
source.transform(trans_init)
draw_registration_result(source, target, np.identity(4))
print("source points:", len(source.points))
print("target points:", len(target.points))
source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)

target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
print("source_down points:", len(source_down.points))
print("target_down points:", len(target_down.points))

source points: 291548
target points: 291301
:: Downsample with a voxel size 0.000.
:: Estimate normal with search radius 0.000.
:: Compute FPFH feature with search radius 0.000.
:: Downsample with a voxel size 0.000.
:: Estimate normal with search radius 0.000.
:: Compute FPFH feature with search radius 0.000.
source_down points: 11293
target_down points: 10092


In [43]:
# o3d.visualization.draw_geometries([source_down])

In [44]:
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, 
        source_fpfh, target_fpfh, 
        True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        ransac_n = 3, 
        checkers = [ #Pruning, points that pass the pruning will be subject to RANSAC
            #Checking if the edeges of source and target are about 0.9 of each other
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9), 
            #Checking if the distance between the points is less than the threshold
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold) 
        ], 
        criteria = o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999)) #max_iter, confidence
    return result.transformation

In [45]:
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print(result_ransac)

:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.000,
   we use a liberal distance threshold 0.000.
[[ 1.03418439e-01  9.90843551e-01 -8.67967948e-02  3.51935160e-06]
 [ 6.33477340e-02  8.05260323e-02  9.94737464e-01 -3.95788075e-05]
 [ 9.92618603e-01 -1.08372576e-01 -5.44398172e-02 -4.27019393e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


Draw results with color

In [47]:
draw_registration_result(source_down, target_down, result_ransac, diff_color=False)